# Only Parallel Data Model

This is the Python Notebook concerning the final version containing the coding for the Only-Parallel (Supervised) version of the program. It is to be completed with the correct format of the Contextual and Style Classifiers in their corresponding parts.

Date of upload: Friday 31th January

Actual Version: 3.0, coherence implemented (Monday 10th February, night)

Precedent Versions : 2.1, 2.0, 1.1, 1.0



## import(s)

In [5]:
from scripts.data_builders.prepare_dataset import prepare_dataset_parallel,string2code,code2string

import math
import torch
import torchvision.datasets as datasets
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
from torchnlp.metrics import get_moses_multi_bleu,get_token_accuracy
from torch.optim import Adam

from pathlib import Path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device = ",device)
print(torch.__version__)

device =  cuda
1.4.0


## Data Pre-processing

In [6]:
train_data, dict_words = prepare_dataset_parallel("data/shakespeare.csv",device,ratio=0.5) #check with shift+tab to look at the data structure
dict_token = {b:a for a,b in dict_words.items()}

Loading ...
- Shakespeare dataset length :  21079
- Corrupted samples (ignored) :  0


## Parameters and Embedding

In [7]:
savepath = Path("data/models/embedding_v1")
embedding = torch.load(savepath,map_location = device)
embedding.weight.requires_grad = False

In [8]:
nb_heads = 4
d_feedforward = 1024
batch_size = 32
dict_size = len(dict_token)

d_embedding = embedding.embedding_dim

## Positional Encoding

In [9]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## Context Classifier (To be filled)

In [10]:
savepath_style = Path("data/models/coherence_classifier_v1_epoch_100")
coherence_classifier = torch.load(savepath_style,map_location = device)
coherence_classifier.requires_grad = False

## Style classifier (To be filled)

In [11]:
savepath_style = Path("data/models/style_classifier_v1")
style_classifier = torch.load(savepath_style,map_location = device)
style_classifier.requires_grad = False

## Model (To be adapted once we have the Context and Style Class For the Embedding)

In [12]:
class ParallelModel(torch.nn.Module):
    def __init__(self,dict_size, d_embedding, nb_heads, d_feedforward):
        super().__init__()
       
        self.embed_layer = torch.nn.Embedding(dict_size+1, d_embedding, padding_idx=dict_size)
        self.positional_layer = PositionalEncoding(d_embedding)
        self.sentence_encoder = torch.nn.TransformerEncoderLayer(d_model = d_embedding, nhead = nb_heads,
                                                    dim_feedforward = d_feedforward)
        self.context_encoder = torch.nn.TransformerEncoderLayer(d_model = d_embedding, nhead = nb_heads,
                                                    dim_feedforward = d_feedforward)
        self.sentence_decoder = torch.nn.TransformerDecoderLayer(d_model = d_embedding, nhead = nb_heads,
                                                    dim_feedforward = d_feedforward)
        self.label_embedding = torch.nn.Embedding(2,768)
        self.padd = dict_size
   
    def _generate_padding_mask(self,x):
        mask = (x == dict_size)
        return mask
   
    def _generate_square_subsequent_mask(self,sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
   
    def forward(self,x,ctx_x,y,label_x):
        device = x.device
        padd_x_mask = self._generate_padding_mask(x).to(device)
        padd_ctx_mask = self._generate_padding_mask(ctx_x).to(device)
        padd_y_mask = self._generate_padding_mask(y).to(device)
       
        mask_x = self._generate_square_subsequent_mask(x.shape[1]).to(device)
        mask_ctx = self._generate_square_subsequent_mask(ctx_x.shape[1]).to(device)
        mask_y = self._generate_square_subsequent_mask(y.shape[1]).to(device)
       
        # Embedding
        x = self.embed_layer(x).transpose(0,1) # token x batch x embedding
        ctx_x = self.embed_layer(ctx_x).transpose(0,1)
       
        # Positional Encoding
        x = self.positional_layer(x)
        ctx_x = self.positional_layer(ctx_x)        
       
        # Encoders
        x_enc = self.sentence_encoder(x,src_mask=mask_x,src_key_padding_mask=padd_x_mask)
        ctx_enc = self.context_encoder(ctx_x,src_mask=mask_ctx,src_key_padding_mask=padd_ctx_mask)
 
        # Linear and Style Mixing
        x_and_ctx = torch.cat((x_enc,ctx_enc),dim = 0)
        label = (1-label_x).reshape((1,x_and_ctx.shape[1])).expand((x_and_ctx.shape[0],x_and_ctx.shape[1])).to(device)
        x_lab = x_and_ctx + self.label_embedding(label)
       
        # Decoder
        padd_mem_mask = torch.cat((padd_x_mask,padd_ctx_mask),1)
        y = self.embed_layer(y)
        y_pos = self.positional_layer(y.transpose(0,1))
        y_pred = self.sentence_decoder(y_pos,x_lab,tgt_mask=mask_y,tgt_key_padding_mask=padd_y_mask,
                                       memory_key_padding_mask=padd_mem_mask)
       
        return(y_pred.transpose(0,1),y)
   
    def translator(self,x,ctx_x,y,label_x):
        device = x.device
        mask_x = self._generate_square_subsequent_mask(x.shape[1]).to(device)
        mask_ctx = self._generate_square_subsequent_mask(ctx_x.shape[1]).to(device)
       
        # Embedding
        x = self.embed_layer(x).transpose(0,1) # token x batch x embedding
        ctx_x = self.embed_layer(ctx_x).transpose(0,1)
       
        # Positional Encoding
        x = self.positional_layer(x)
        ctx_x = self.positional_layer(ctx_x)        
       
        # Encoders
        x_enc = self.sentence_encoder(x,mask_x)
        ctx_enc = self.context_encoder(ctx_x,mask_ctx)
       
        # Linear and Style Mixing
        x_and_ctx = torch.cat((x_enc,ctx_enc),dim = 0)
        label = (1-label_x).reshape((1,x_and_ctx.shape[1])).expand((x_and_ctx.shape[0],x_and_ctx.shape[1])).to(device)
        x_lab = x_and_ctx + self.label_embedding(label)
        return(x_lab)

## Training  (To be adapted once we have the Context and Style Class For the Embedding)

In [13]:
# Definition of the model(s)

model = ParallelModel(dict_size, d_embedding, nb_heads, d_feedforward).to(device)
model.embed_layer = embedding
model.embed_layer.weight.requires_grad = False

In [14]:
# Information concerning the Training optimizer

decoder_linear = torch.nn.Linear(d_embedding,dict_size).to(device)
softmax_layer = torch.nn.LogSoftmax(dim = 2).to(device)

params = list(model.parameters()) + list(decoder_linear.parameters())#+ list(context_encoder.parameters()) + 
                                  #list(linear_context.parameters) + list(sentence_decoder.parameters())

l_r = 5e-4
optimizer=Adam(params,lr=l_r)

#Weights of the losses
l1=1 #
l2=1
l3=1

In [15]:
# Losses 
#loss_seq2seq = torch.nn.SmoothL1Loss(reduction='mean') #Contextual Seq2Seq Loss
#loss_seq2seq = torch.nn.KLDivLoss(reduction = 'mean')
loss_seq2seq = torch.nn.CrossEntropyLoss(reduction = 'mean',ignore_index=dict_size)

In [16]:
nb_epoch = 100
writer = SummaryWriter("data/runs/parallel_model_v0")
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                            shuffle=True,collate_fn=train_data.collate)
n = len(train_data.x) // batch_size

for epoch in range(1,nb_epoch+1):
    total_loss = total_seq = total_style = total_coh = total_accuracy = total_BLEU = 0
    i = 0
    
    for x,y, ctx_x, ctx_y_1,ctx_y_2, label_x,len_y in train_loader:
        i += 1
        if i==n:
            break
        optimizer.zero_grad()
        
        x,_ = model.forward(x,ctx_x,y,label_x) #Output still embedded
        y_pred_dist = decoder_linear(x)
        
        
        # Seq2Seq Loss with Embedding
        #y_hot = torch.nn.functional.one_hot(y).float()
        #y_hot[y_hot==dict_size] = 0.
        
        loss_seq = loss_seq2seq(y_pred_dist.reshape(-1,dict_size),y.reshape(-1))
        
        # Style Loss
        loss_sty,_ = style_classifier.forward(inputs_embeds=x,attention_mask=(y != dict_size).int(),labels=(1-label_x).to(device))
        
        
        # Coherence Loss
        ctx_y = torch.cat([embedding(ctx_y_1),x,embedding(ctx_y_2)],dim=1)
        mask_coh = torch.cat([(ctx_y_1 != dict_size).int(),
                              ((y != dict_size)*(y != 0)*(y != 1)).int(),
                              (ctx_y_2 != dict_size).int()],dim=1).to(device)
        loss_coh,_ = coherence_classifier(inputs_embeds=ctx_y,attention_mask=mask_coh,labels=torch.LongTensor([1]*batch_size).to(device))
        
        
        # Total Loss
        loss = l1 * loss_seq + l2 * loss_sty + l3 *  loss_coh

        # Step
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_seq += loss_seq.item()
        total_style += loss_sty.item()
        total_coh += loss_coh.item()
        
        pred = torch.argmax(softmax_layer(y_pred_dist),dim=2)
        hypotheses = [code2string(item[:len_],dict_token,no_pad=True) for item,len_ in zip(pred,len_y)]
        references = [code2string(item[:len_],dict_token,no_pad=True) for item,len_ in zip(y,len_y)]
        accuracy,_,_ = get_token_accuracy(y,pred)
        total_accuracy += accuracy
        BLEU = get_moses_multi_bleu(hypotheses,references,lowercase=True)
        total_BLEU += BLEU if BLEU else 0

    #Vizualization and saving model
        if ((i-1) % 94 == 0):
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'total loss {:5.2f} | seq2seq loss {:5.2f} | '
                  'style loss {:5.2f} | coherence loss {:5.2f}'
                  '| total accuracy {:5.2f} | total BLEU {:5.2f}'.format(
                    epoch, i, n,total_loss/i,total_seq/i,total_style/i,total_coh/i,total_accuracy/i,total_BLEU/i))
    print('-' * 110)
    writer.add_scalar('train_loss',total_loss/(n-1),epoch)
    writer.add_scalar('train_loss_seq',total_seq/(n-1),epoch)
    writer.add_scalar('train_loss_style',total_style/(n-1),epoch)
    writer.add_scalar('train_loss_coh',total_coh/(n-1),epoch)
    writer.add_scalar('train_word_accuracy',total_accuracy/(n-1),epoch)
    writer.add_scalar('train_BLEU',total_BLEU/(n-1),epoch)
    if (epoch%5==0):
        torch.save(model,"data/models/parallel_model_v0_epoch_"+str(epoch))
        torch.save(decoder_linear,"data/modelsparallel_decoder_linear_v0_epoch_"+str(epoch))

| epoch   1 |     1/  658 batches | total loss 14.47 | seq2seq loss  9.95 | style loss  2.01 | coherence loss  2.51| total accuracy  0.00 | total BLEU  0.00
| epoch   1 |    95/  658 batches | total loss  4.49 | seq2seq loss  4.33 | style loss  0.10 | coherence loss  0.06| total accuracy  0.17 | total BLEU 18.25
| epoch   1 |   189/  658 batches | total loss  2.96 | seq2seq loss  2.88 | style loss  0.05 | coherence loss  0.03| total accuracy  0.24 | total BLEU 42.83
| epoch   1 |   283/  658 batches | total loss  2.29 | seq2seq loss  2.23 | style loss  0.04 | coherence loss  0.03| total accuracy  0.27 | total BLEU 54.73
| epoch   1 |   377/  658 batches | total loss  1.90 | seq2seq loss  1.85 | style loss  0.03 | coherence loss  0.02| total accuracy  0.29 | total BLEU 62.12
| epoch   1 |   471/  658 batches | total loss  1.64 | seq2seq loss  1.60 | style loss  0.02 | coherence loss  0.02| total accuracy  0.30 | total BLEU 67.17
| epoch   1 |   565/  658 batches | total loss  1.45 | seq

multi-bleu.perl script returned non-zero exit code
b'\ngzip: /tmp/tmpw2f0v0gz: not in gzip format\nERROR: could not find reference file /tmp/tmpw2f0v0gz at /tmp/tmp10lbee28 line 32.\n'


--------------------------------------------------------------------------------------------------------------
| epoch   3 |     1/  658 batches | total loss  0.22 | seq2seq loss  0.21 | style loss  0.00 | coherence loss  0.01| total accuracy  0.19 | total BLEU 94.89
| epoch   3 |    95/  658 batches | total loss  0.14 | seq2seq loss  0.13 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 97.34
| epoch   3 |   189/  658 batches | total loss  0.14 | seq2seq loss  0.13 | style loss  0.00 | coherence loss  0.01| total accuracy  0.34 | total BLEU 97.42
| epoch   3 |   283/  658 batches | total loss  0.14 | seq2seq loss  0.14 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 97.34
| epoch   3 |   377/  658 batches | total loss  0.15 | seq2seq loss  0.14 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 97.21
| epoch   3 |   471/  658 batches | total loss  0.14 | seq2seq loss  0.14 | style loss  0.00 | coherence

/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ParallelModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type PositionalEncoding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| epoch   6 |     1/  658 batches | total loss  0.01 | seq2seq loss  0.01 | style loss  0.00 | coherence loss  0.01| total accuracy  0.28 | total BLEU 100.00
| epoch   6 |    95/  658 batches | total loss  0.02 | seq2seq loss  0.02 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.68
| epoch   6 |   189/  658 batches | total loss  0.02 | seq2seq loss  0.02 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.72
| epoch   6 |   283/  658 batches | total loss  0.02 | seq2seq loss  0.02 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.69
| epoch   6 |   377/  658 batches | total loss  0.02 | seq2seq loss  0.02 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.67
| epoch   6 |   471/  658 batches | total loss  0.02 | seq2seq loss  0.02 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.70
| epoch   6 |   565/  658 batches | total loss  0.02 | se

...

| epoch  94 |   377/  658 batches | total loss  0.01 | seq2seq loss  0.00 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.28
| epoch  94 |   471/  658 batches | total loss  0.01 | seq2seq loss  0.00 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.34
| epoch  94 |   565/  658 batches | total loss  0.01 | seq2seq loss  0.00 | style loss  0.00 | coherence loss  0.01| total accuracy  0.36 | total BLEU 99.40
--------------------------------------------------------------------------------------------------------------
| epoch  95 |     1/  658 batches | total loss  0.01 | seq2seq loss  0.00 | style loss  0.00 | coherence loss  0.00| total accuracy  0.37 | total BLEU 100.00
| epoch  95 |    95/  658 batches | total loss  0.01 | seq2seq loss  0.00 | style loss  0.00 | coherence loss  0.01| total accuracy  0.36 | total BLEU 99.89
| epoch  95 |   189/  658 batches | total loss  0.01 | seq2seq loss  0.00 | style loss  0.00 | coherenc

multi-bleu.perl script returned non-zero exit code
b'\ngzip: /tmp/tmpa7bujjgz: not in gzip format\nERROR: could not find reference file /tmp/tmpa7bujjgz at /tmp/tmpk87qa6g7 line 32.\n'


--------------------------------------------------------------------------------------------------------------
| epoch 100 |     1/  658 batches | total loss  0.01 | seq2seq loss  0.00 | style loss  0.00 | coherence loss  0.01| total accuracy  0.22 | total BLEU 100.00
| epoch 100 |    95/  658 batches | total loss  0.01 | seq2seq loss  0.01 | style loss  0.00 | coherence loss  0.01| total accuracy  0.36 | total BLEU 99.57
| epoch 100 |   189/  658 batches | total loss  0.01 | seq2seq loss  0.01 | style loss  0.00 | coherence loss  0.01| total accuracy  0.36 | total BLEU 99.67
| epoch 100 |   283/  658 batches | total loss  0.02 | seq2seq loss  0.01 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.32
| epoch 100 |   377/  658 batches | total loss  0.02 | seq2seq loss  0.01 | style loss  0.00 | coherence loss  0.01| total accuracy  0.36 | total BLEU 99.39


multi-bleu.perl script returned non-zero exit code
b'\ngzip: /tmp/tmp60e85tgz: not in gzip format\nERROR: could not find reference file /tmp/tmp60e85tgz at /tmp/tmpp3yhs3hq line 32.\n'


| epoch 100 |   471/  658 batches | total loss  0.02 | seq2seq loss  0.01 | style loss  0.00 | coherence loss  0.01| total accuracy  0.35 | total BLEU 99.24
| epoch 100 |   565/  658 batches | total loss  0.02 | seq2seq loss  0.01 | style loss  0.00 | coherence loss  0.01| total accuracy  0.36 | total BLEU 99.34
--------------------------------------------------------------------------------------------------------------


In [ ]:
# reduction = 'sum' l_r = 5e-4
from matplotlib import pyplot as plt
import numpy as np
rango = np.arange(len(loss_graph))
plt.figure(1,figsize=(16,16))
plt.subplot(231)
plt.plot(rango,loss_graph)
plt.title("Total Loss")
plt.grid()
plt.subplot(234)
plt.plot(rango,loss_graph)
plt.xscale('log')
plt.yscale('log')
plt.title("Total Loss Log")
plt.grid()
plt.subplot(232)
plt.plot(rango,loss_seq_list)
plt.title("Seq Loss")
plt.grid()
plt.subplot(235)
plt.plot(rango,loss_seq_list)
plt.xscale('log')
plt.yscale('log')
plt.title("Seq Loss Log")
plt.grid()
plt.subplot(233)
plt.plot(rango,loss_style_list)
plt.title("Style Loss")
plt.grid()
plt.subplot(236)
plt.plot(rango,loss_style_list)
plt.title("Style Loss Log")
plt.xscale('log')
plt.yscale('log')
plt.grid()
plt.show()

In [ ]:
x,y,ctx_x,_,_,label_x,_ = train_data[0]
if(label_x == 1):
    print("Original Style : Shakespearian")
else:
    print("Original Style : Modern")
print("Original Phrase",code2string(x,dict_token))
print("Target phrase",code2string(y,dict_token))

In [23]:
    #Generation de phrase 
with torch.no_grad():
        h_t = model.translator(x.unsqueeze(0),ctx_x.unsqueeze(0),y.unsqueeze(0),torch.tensor([label_x]).unsqueeze(0)).to(device)
        phrase = torch.tensor([[0]]).to(device)
        print("Starting with: ",end='')
        for p in phrase:
            print(dict_token[p.item()],end=' ')
        print("")
        i = 0
        limit = 20
        flag = False
        while(not(flag) and i != limit):
            #mask = model._generate_square_subsequent_mask(phrase.shape[1])
            y_aux = model.embed_layer(phrase)
            y_pos = model.positional_layer(y_aux)
            y_pred = model.sentence_decoder(y_pos,h_t).transpose(0,1)
            print("avant decodeur : ")
            print(y_pred)
            y_pred = decoder_linear(y_pred)
            y_pred = torch.argmax(softmax_layer(y_pred),dim = 2)
            print("après decodeur : ")
            print(y_pred)
            phrase = torch.cat((phrase,y_pred[:,-1].reshape((1,1))),0)
            i += 1
            flag = (y_pred[:,-1].item() == 1)
        print("Produced phrase: ",end='')
        for p in phrase:
            print(dict_token[p.item()],end=' ')
        print("")
            

Starting with: <SOS> 
avant decodeur : 
tensor([[[-2.8566e+00,  1.3631e+00, -5.2943e-01, -5.4730e-01,  7.3688e-02,
           1.4968e+00, -6.6304e-01, -1.7540e-01,  2.1892e-01,  5.0551e-01,
          -8.9138e-01,  1.3394e+00,  1.0534e-01,  1.1348e-01,  1.4155e+00,
           2.4190e-01, -2.6258e-01,  1.4768e-01, -1.0666e-01,  4.5948e-01,
          -6.0964e-01,  9.6347e-01,  5.5408e-01, -1.9438e+00, -2.2467e-01,
          -5.1118e-01,  6.1061e-01,  2.6865e-01, -3.0314e-01, -2.7878e-02,
          -5.4147e-02,  1.3164e+00,  1.9417e-02, -6.0739e-01, -4.0599e-01,
           2.3132e+00,  1.1610e-01, -2.8582e+00, -5.7736e-01, -9.5192e-01,
           1.3297e-01, -3.1089e-01,  9.7992e-01,  2.1851e-01, -9.1420e-01,
          -7.5019e-01, -6.6953e-01,  1.4033e+00,  3.9269e-01,  9.4089e-01,
           1.0336e+00,  6.3175e-01,  1.5644e-01,  6.3677e-01,  2.4527e-01,
          -1.9399e-01,  4.3632e-01,  1.2974e+00,  4.7962e-02, -1.2857e+00,
           1.9835e-01, -9.3711e-01,  2.1754e-01,  1.6095e+00

In [15]:
label_x.shape

torch.Size([32])

In [19]:
hypotheses

['<SOS> here they’ll be , man . <EOS>',
 '<SOS> this dream has been interpreted all wrong . <EOS>',
 '<SOS> well , that’s what you get to do . <EOS>',
 '<SOS> it might happen that these strange signs , the unusual terror of this night , and the servant of his lark’s will keep him away from the capitol today . <EOS>',
 '<SOS> he received them of him that brought them . <EOS>',
 '<SOS> that you did fear is done . <EOS>',
 '<SOS> tis not to me she speaks . <EOS>',
 '<SOS> will your grace command me any service to the world’s end ? <EOS>',
 '<SOS> and so doth mine . <EOS>',
 '<SOS> i was too young at the time to appreciate her value , but now i know her . <EOS>',
 '<SOS> i know no ways to vomit it in love , but directly to say , i love you . then if you urge me farther than to say , do you , in faith ? i wear out my suit . <EOS>',
 '<SOS> will it eat me ? <EOS>',
 '<SOS> his majesty is so concerned about my personal safety that he has ordered them to conduct me to the you’re being arrested

In [18]:
references

['<SOS> here they’ll be , man . <EOS>',
 '<SOS> this dream has been interpreted all wrong . <EOS>',
 '<SOS> well , that’s what you get to do . <EOS>',
 '<SOS> it might happen that these strange signs , the unusual terror of this night , and the urgings of his fortune-tellers will keep him away from the capitol today . <EOS>',
 '<SOS> he received them of him that brought them . <EOS>',
 '<SOS> that you did fear is done . <EOS>',
 '<SOS> tis not to me she speaks . <EOS>',
 '<SOS> will your grace command me any service to the world’s end ? <EOS>',
 '<SOS> and so doth mine . <EOS>',
 '<SOS> i was too young at the time to appreciate her value , but now i know her . <EOS>',
 '<SOS> i know no ways to mince it in love , but directly to say , i love you . then if you urge me farther than to say , do you , in faith ? i wear out my suit . <EOS>',
 '<SOS> will it eat me ? <EOS>',
 '<SOS> his majesty is so concerned about my personal safety that he has ordered them to conduct me to the you’re being

In [24]:
5%1

0